In [1]:
import os
import gensim
from tqdm import tqdm
import numpy as np
import keras
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from collections import Counter
import re
import string
import nltk
from keras.utils import np_utils,pad_sequences

2022-07-25 20:09:15.799430: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-25 20:09:15.802440: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-25 20:09:15.802452: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(os.listdir("word2vec_dataset/word2vec_dataset/review_hotels/data"))

['chicago', 'las-vegas', 'london', 'dubai', 'new-york-city', 'shanghai', 'new-delhi', 'san-francisco', 'montreal', '.DS_Store', 'beijing']


In [3]:
no_date_text=[]
path="word2vec_dataset/word2vec_dataset/review_hotels/data"
for folders in os.listdir(path):
    if not folders[0].isalnum():
        continue
    i=0
    for filename in os.listdir(path + '/' + folders):
        with open(os.path.join(path + '/' + folders,filename),'r',errors='ignore') as f:
            for sentence in f.readlines():
                try:
                    no_date_text.append(sentence[sentence.index('\t')+1:])
                except:
                    continue

In [4]:
stopwords = nltk.corpus.stopwords.words('english')
lemma=WordNetLemmatizer()
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [lemma.lemmatize(word) for word in tokens if word not in stopwords and word.isalpha()]
    return text

In [5]:
word_lists = []
all_words=Counter()
maxlen=30

for i,text in tqdm(enumerate(no_date_text)):
    no_date_text[i] = clean_text(text)[:maxlen+1]
    for word in no_date_text[i]:
        all_words[word]+=1

255138it [01:40, 2528.18it/s]


In [6]:
k=0
uniq_words=[]
for w,i in sorted(all_words.items(),key=lambda x:x[1],reverse=True):
    if k>9999:
        break
    uniq_words.append(w)
    k+=1

In [7]:
unique_word_dict = {}
for i, word in tqdm(enumerate(uniq_words)):
    unique_word_dict.update({
        word: i
    })

10000it [00:00, 1460361.41it/s]


In [8]:
for j in tqdm(range(len(no_date_text))):
    no_date_text[j]=[unique_word_dict[word] for word in no_date_text[j] if word in uniq_words]

100%|█████████████████████████████████| 255138/255138 [01:12<00:00, 3542.14it/s]


In [9]:
y=[np_utils.to_categorical(2, 10000),np_utils.to_categorical(3, 10000)]
np.asarray(y).shape

(2, 10000)

In [19]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, word_lists, unique_word_dict,window_size, batch_size=128,shuffle=False):
        'Initialization'
        self.word_lists = word_lists
        self.batch_size = batch_size
        self.unique_word_dict = unique_word_dict
        self.window_size=window_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.word_lists) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.word_lists[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.word_lists))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, word_lists_temp):
        'Generates data containing batch_size samples'
        # Initialization
        n=len(self.unique_word_dict)
        x=[]
        y=[]
        
        context_length = self.window_size*2
        for words in word_lists_temp:
            n1=len(words)
            for index, word in enumerate(words):           
                start = index - self.window_size
                end = index + self.window_size + 1
            
                context_words=[words[i] for i in range(start, end) if 0 <= i < n1 and i != index]
                y.append(np_utils.to_categorical(word, n))

                for i in range(context_length-len(context_words)):
                    context_words.append(0)
                
                x.append(context_words)

        return np.asarray(x),np.asarray(y)

In [21]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

n=len(unique_word_dict)
embed_size=100
win_size=4
data_generator=DataGenerator(no_date_text,unique_word_dict,window_size=win_size,batch_size=128)

model = Sequential()
model.add(Embedding(input_dim=n, output_dim=embed_size, input_length=win_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
model.add(Dense(n, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit_generator(
    generator=data_generator,
    use_multiprocessing=True,
    workers=6,
    epochs=4
    )


/tmp/ipykernel_45221/611703865.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/4
1993/1993 [==============================] - 868s 435ms/step - loss: 6.8927
Epoch 2/4
1993/1993 [==============================] - 877s 439ms/step - loss: 6.2223
Epoch 3/4
1993/1993 [==============================] - 876s 439ms/step - loss: 5.9522
Epoch 4/4
1993/1993 [==============================] - 903s 451ms/step - loss: 5.7969


In [29]:
model.save('finalembedd')

INFO:tensorflow:Assets written to: finalembedd/assets


In [22]:
weights = model.get_weights()[0]

embedding_dict = {}
for word in uniq_words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })

In [23]:
pol=embedding_dict['polite']
dirt=embedding_dict['dirty']
fra=embedding_dict['france']
sho=embedding_dict['shocked']

In [24]:
from numpy.linalg import norm
def cossim(x,y):
    return np.dot(x,y)/norm(x)*norm(y)
def top(x):
    tops=[]
    for w,i in embedding_dict.items():
        tops.append([cossim(i,x),w])
    tops.sort(reverse=True)
    return tops[:10]

In [25]:
print(top(pol))

[[29.635332, 'polite'], [28.015253, 'courteous'], [26.98552, 'cordial'], [26.162766, 'curtious'], [25.99898, 'considerate'], [25.762526, 'freindly'], [25.252836, 'curteous'], [25.22877, 'personable'], [25.153065, 'friendlythe'], [25.011658, 'friendly']]


In [26]:
print(top(dirt))

[[40.98794, 'dirty'], [38.460083, 'filthy'], [37.44034, 'smelly'], [36.516823, 'gross'], [36.508984, 'nasty'], [35.86055, 'moldy'], [35.842808, 'unclean'], [35.351227, 'dusty'], [35.21275, 'disgustingly'], [35.139454, 'stinky']]


In [27]:
print(top(fra))

[[6.682072, 'france'], [5.254983, 'canada'], [5.065885, 'rajasthan'], [5.0201097, 'spain'], [4.8715787, 'edinburgh'], [4.8692746, 'whirlwind'], [4.760905, 'ca'], [4.7048316, 'agra'], [4.697535, 'singapore'], [4.6870914, 'ireland']]


In [28]:
print(top(sho))

[[16.35955, 'shocked'], [13.470577, 'horrified'], [13.263062, 'dismayed'], [12.796984, 'relieved'], [12.638424, 'uncertain'], [12.586542, 'unknown'], [12.508048, 'shock'], [12.492645, 'aback'], [12.316586, 'fear'], [12.043133, 'dissappointed']]
